In [1]:
import spf
import torch

In [5]:
X = spf.utils.grid[.05:1:.05,.05:1:.05].reshape(-1, 2)
V = spf.utils.sqnorm
sf = spf.EulerianSPF(X, V, .1)
sf.set_optimizer(spf.optimizers.FrankWolfe())
µ0 = torch.zeros(X.size(0))
µ0[-1] = 1
sf.integrate(µ0, torch.linspace(0, 1, 500), print_progress=True, max_sinkhorn_steps=1, max_optim_steps=1)

 SJKO step 499...

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        [6.6667e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         3.3333e-01],
        [2.2222e-01, 6.6667e-01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.1111e-01],
        ...,
        [1.5487e-08, 4.6461e-08, 1.3938e-07,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [5.1623e-09, 1.5487e-08, 4.6461e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.7208e-09, 5.1623e-09, 1.5487e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])

torch.Size([19, 19, 2])